In [23]:
ASSETS_GROUP = {'assets': ['yfi', 'alcx', 'farm', 'auto', 'pickle', 'ren', 'badger'], 'cex_platform': ['bnb', 'okb', 'ftt', 'ht', 'leo', 'vgx', 'btmx', 'mdx', 'mx', 'woo'], 'DAO': ['dcr', 'ant', 'dafi'], 'derivative_synthesis': ['hegic', 'snx', 'uma', 'mir', 'lina'], 'dex': ['uni', 'cake', 'sushi', 'zrx', 'bnt', '1inch', 'crv', 'lrc', 'knc', 'bal', 'dodo', 'srm', 'rep', 'swap', 'perp', 'lon', 'any', 'kex'], 'insurance': ['nxm', 'wnxm', 'helmet', 'armor', 'yam'], 'lending': ['aave', 'comp', 'mkr', 'xvs', 'alpha', 'kava', 'mft', 'cream', 'uft', 'cel'], 'NFT': ['chz', 'enj', 'mana', 'flow', 'sand', 'ern', 'alice', 'rari', 'matic', 'waxp', 'tvk', 'cws', 'axs'], 'oracle': ['link', 'band', 'api3', 'trb', 'nest'], 'stablecoin': ['luna', 'rsr', 'ampl', 'esd', 'fxs', 'bank', 'bas', 'bac'], 'public_chain': ['btc', 'eth', 'ada', 'vet', 'miota', 'xtz', 'neo', 'algo', 'hbar', 'stx', 'near', 'icx', 'ont', 'qtum', 'one', 'iost', 'ckb', 'stmx', 'vtho', 'ewt', 'orbs', 'lyxe', 'chr', 'lsk', 'waves', 'zil', 'atom', 'sol', 'AVAX', 'egld', 'qkc', 'xem', 'ftm'], 'storage': ['fil', 'ar', 'storj', 'sc', 'cru'], 'payment': ['cro', 'omg', 'npxs', 'nano', 'sxp', 'utk', 'coti', 'pac', 'tel', 'meta', 'nim', 'xlm'],  'data': ['dent', 'nmr', 'bdp', 'ocean', 'btt', 'grt'], 'dot': ['ksm', 'pols', 'fis', 'reef', 'edg', 'dot', 'ring'], 'privacy': ['xmr', 'dash', 'zec', 'xvg', 'rose', 'torn']}

In [25]:
from datetime import datetime, timedelta
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import numpy as np
import pandas as pd
import seaborn as sns
import json
import time

def fetch_historical_data_to_df(url_pattern, assets_group, start_date, end_date, key_index):
    parameters = {
        'start':start_date,
        'end':end_date,
        'interval':'1d'
    }
    headers = {
        'Accepts': 'application/json',
    }

    session = Session()
    session.headers.update(headers)

    ## subtract 1 day to make the index consistent for different libraries
    start_date_delta = (datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=1)).date()
    end_date_delta = (datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=1)).date()

    days = pd.date_range(start_date_delta, end_date_delta, freq='D').date

    df = pd.DataFrame(days, columns=['Date'])
    df = df.set_index('Date', drop=True)

    for key, assets in assets_group.items():
        for asset in assets:
            try:
                response = session.get(url_pattern.format(assetKey=asset), params=parameters)

                time.sleep(8)

                if response.status_code != 200:
                    print('status code: %d for %s.' % (response.status_code, asset))
                    continue

                historical_data_json = json.loads(response.text)

                historical_data = historical_data_json['data']['values']

                if not historical_data:
                    print('no data for %s.' % asset)
                    continue

                value_history = {str(day): 0 for day in days}
                for value in historical_data:
                    date = str(datetime.fromtimestamp(value[0]/1e3).date())
                    if not date in value_history:
                        print("fatal error")
                    value_history[date] = value[key_index]

                df[asset] = pd.Series(value_history).values                
            except (ConnectionError, Timeout, TooManyRedirects) as e:
                print(e)

    return df

In [11]:
CLOSE_PRICE_IDX = 4

START_DATE = '2020-10-15'
END_DATE ='2021-04-17'

df = fetch_historical_data_to_df(
  url_pattern='https://data.messari.io/api/v1/assets/{assetKey}/metrics/price/time-series',
  assets_group=ASSETS_GROUP,
  start_date=START_DATE,
  end_date=END_DATE,
  key_index=CLOSE_PRICE_IDX)

df.to_pickle('data/price_history.pkl')            

no price for auto.
no price for pickle.
no price for vgx.
status code: 404 for btmx.
no price for mdx.
status code: 404 for mx.
status code: 404 for woo.
status code: 404 for dafi.
status code: 404 for any.
no price for nxm.
status code: 404 for helmet.
status code: 404 for ern.
no price for bac.
status code: 404 for lyxe.
no price for cru.
no price for coti.
status code: 404 for nim.
no price for ocean.
no price for btt.
no price for fis.
status code: 404 for torn.


In [26]:
MCAP_INDEX = 1

START_DATE = '2020-10-15'
END_DATE ='2021-04-17'

df = fetch_historical_data_to_df(
  url_pattern='https://data.messari.io/api/v1/assets/{assetKey}/metrics/mcap.circ/time-series',
  assets_group=ASSETS_GROUP,
  start_date=START_DATE,
  end_date=END_DATE,
  key_index=MCAP_INDEX)

df.to_pickle('data/mcap_history.pkl')      

no data for alcx.
no data for auto.
no data for pickle.
no data for badger.
no data for vgx.
status code: 404 for btmx.
no data for mdx.
status code: 404 for mx.
status code: 404 for woo.
status code: 404 for dafi.
no data for mir.
no data for cake.
no data for 1inch.
no data for dodo.
no data for swap.
no data for lon.
status code: 404 for any.
no data for kex.
no data for nxm.
no data for wnxm.
status code: 404 for helmet.
no data for armor.
no data for yam.
no data for xvs.
no data for uft.
no data for flow.
status code: 404 for ern.
no data for alice.
no data for tvk.
no data for cws.
no data for axs.
no data for api3.
no data for esd.
no data for fxs.
no data for bank.
no data for bas.
no data for bac.
no data for one.
status code: 404 for lyxe.
no data for cru.
no data for coti.
no data for pac.
status code: 404 for nim.
no data for bdp.
no data for ocean.
no data for btt.
no data for grt.
no data for pols.
no data for fis.
no data for reef.
no data for ring.
no data for rose.
st